# Facial Detection

The goal of this python file is to fully train a model to recognize the age, gender, and emotion of a face that it detects either from an uplaoded image or live camera detection

#### 1. Imports and basic configuration

In [83]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split

print("TensorFlow version:", tf.__version__)

# Image + training settings 
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS_HEADS = 10
EPOCHS_FINETUNE = 10

TensorFlow version: 2.20.0


#### 2. Load merged CSV and compute class counts

In [84]:
csv_path = r"C:\Users\bensa\merged_dataset.csv"
df = pd.read_csv(csv_path)  # columns: image_path, age, gender, emotion [file:1]

print(df.head())

# Analyze the distribution of age, gender, and emotion labels/ class distributions
print("\nAge value counts:\n", df["age"].value_counts())
print("\nGender value counts:\n", df["gender"].value_counts())
print("\nEmotion value counts:\n", df["emotion"].value_counts())

                                          image_path  age  gender  emotion
0  source_data/UTK-Face/part3/27_0_1_201701201338...    2       0       -1
1  source_data/UTK-Face/part3/24_0_3_201701191655...    2       0       -1
2  source_data/UTK-Face/part3/8_1_0_2017011715460...    0       1       -1
3  source_data/UTK-Face/part3/85_1_0_201701202226...    6       1       -1
4  source_data/UTK-Face/part3/26_1_0_201701191929...    2       1       -1

Age value counts:
 age
 2    12339
-1     5102
 0     4823
 3     4754
 6     2936
 4     2460
 5     2395
 1     1564
Name: count, dtype: int64

Gender value counts:
 gender
0    20347
1    16026
Name: count, dtype: int64

Emotion value counts:
 emotion
-1    24102
 4     4772
 7     2524
 5     1982
 1     1290
 3      717
 6      705
 2      281
Name: count, dtype: int64


In [93]:
# If image paths in CSV are relative, prepend a root dir
root_dir = r"C:/Users/bensa"   # change this if images are under a specific folder

print("\nOriginal image paths:\n", df["image_path"].head())
df["image_path"] = df["image_path"].apply(lambda p: os.path.join(root_dir, p))

print("\nUpdated image paths:\n")
print(df["image_path"].head())

# print("Before:\n", df["image_path"].head())        # before applying join
# df["image_path"] = df["image_path"].apply(lambda p: os.path.join(root_dir, p))
# print("After:\n", df["image_path"].head())         # after applying join


# Compute number of classes
NUM_AGE_CLASSES = int(df["age"].max() + 1)          # ages are binned 0..6
NUM_GENDER_CLASSES = int(df["gender"].max() + 1)    # usually 2

valid_emotions = df[df["emotion"] >= 0]["emotion"].unique()
NUM_EMOTION_CLASSES = int(valid_emotions.max() + 1)

print("NUM_AGE_CLASSES:", NUM_AGE_CLASSES, type(NUM_AGE_CLASSES))
print("NUM_GENDER_CLASSES:", NUM_GENDER_CLASSES, type(NUM_GENDER_CLASSES))
print("NUM_EMOTION_CLASSES:", NUM_EMOTION_CLASSES, type(NUM_EMOTION_CLASSES))




Original image paths:
 0    C:/Users/bensa\source_data/UTK-Face/part3/27_0...
1    C:/Users/bensa\source_data/UTK-Face/part3/24_0...
2    C:/Users/bensa\source_data/UTK-Face/part3/8_1_...
3    C:/Users/bensa\source_data/UTK-Face/part3/85_1...
4    C:/Users/bensa\source_data/UTK-Face/part3/26_1...
Name: image_path, dtype: str

Updated image paths:

0    C:/Users/bensa\source_data/UTK-Face/part3/27_0...
1    C:/Users/bensa\source_data/UTK-Face/part3/24_0...
2    C:/Users/bensa\source_data/UTK-Face/part3/8_1_...
3    C:/Users/bensa\source_data/UTK-Face/part3/85_1...
4    C:/Users/bensa\source_data/UTK-Face/part3/26_1...
Name: image_path, dtype: str
NUM_AGE_CLASSES: 7 <class 'int'>
NUM_GENDER_CLASSES: 2 <class 'int'>
NUM_EMOTION_CLASSES: 8 <class 'int'>


#### Train and Validation Split

In [89]:
# Split the dataset into training and validation sets
train_df, val_Df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

print(f"Training samples (80%): {len(train_df)}, Validation samples (20%): {len(val_Df)}")

Training samples (80%): 29098, Validation samples (20%): 7275


#### 4. Building tf.data pipelines with masking

Key idea: each sample returns image + labels + masks so losses can ignore missing emotion.

In [94]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

print("Training samples:", len(train_df))
print("Validation samples:", len(val_df))


Training samples: 29098
Validation samples: 7275


In [95]:
IMAGE_SIZE = 224
BATCH_SIZE = 32

def load_and_preprocess(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    img = tf.cast(img, tf.float32)  # 0..255
    img = keras.applications.efficientnet.preprocess_input(img)
    return img

def make_dataset(df_in, shuffle=True):
    image_paths = df_in["image_path"].values.astype(str)
    ages = df_in["age"].values.astype("int32")
    genders = df_in["gender"].values.astype("int32")
    emotions = df_in["emotion"].values.astype("int32")

    ds = tf.data.Dataset.from_tensor_slices((image_paths, ages, genders, emotions))

    def _map_fn(image_path, age, gender, emotion):
        img = load_and_preprocess(image_path)

        # masks: 1 if label exists, 0 if missing (-1)
        has_age = tf.cast(age >= 0, tf.float32)
        has_gender = tf.cast(gender >= 0, tf.float32)
        has_emotion = tf.cast(emotion >= 0, tf.float32)

        # replace missing labels with dummy 0 (masked out anyway)
        age = tf.where(age >= 0, age, 0)
        gender = tf.where(gender >= 0, gender, 0)
        emotion = tf.where(emotion >= 0, emotion, 0)

        labels = {
            "age": age,
            "gender": gender,
            "emotion": emotion,
            "has_age": has_age,
            "has_gender": has_gender,
            "has_emotion": has_emotion
        }
        return img, labels

    ds = ds.map(_map_fn, num_parallel_calls=tf.data.AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(buffer_size=min(len(df_in), 2048), reshuffle_each_iteration=True)

    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = make_dataset(train_df, shuffle=True)
val_ds = make_dataset(val_df, shuffle=False)

# quick sanity check
batch_imgs, batch_labels = next(iter(train_ds))
print("Batch images:", batch_imgs.shape)
print("Label keys:", batch_labels.keys())


Batch images: (32, 224, 224, 3)
Label keys: dict_keys(['age', 'gender', 'emotion', 'has_age', 'has_gender', 'has_emotion'])


#### 5. EfficientNetB0 multi-task model

In [96]:
def build_efficientnet_multitask(num_age, num_gender, num_emotion):
    inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name="image")

    base_model = keras.applications.EfficientNetB0(
        include_top=False,
        weights="imagenet",
        input_tensor=inputs,
        pooling="avg"
    )
    base_model.trainable = False

    x = base_model.output
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    age_output = layers.Dense(num_age, activation="softmax", name="age")(x)
    gender_output = layers.Dense(num_gender, activation="softmax", name="gender")(x)
    emotion_output = layers.Dense(num_emotion, activation="softmax", name="emotion")(x)

    model = keras.Model(inputs=inputs, outputs=[age_output, gender_output, emotion_output])
    return model, base_model

model, base_model = build_efficientnet_multitask(
    NUM_AGE_CLASSES, NUM_GENDER_CLASSES, NUM_EMOTION_CLASSES
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image (InputLayer)  │ (None, 224, 224,  │          0 │ -                 │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_18        │ (None, 224, 224,  │          0 │ image[0][0]       │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_9     │ (None, 224, 224,  │          7 │ rescaling_18[0][… │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_19        │ (None, 224, 224,  │          0 │ normalization_9[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_19[0][… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,381,876 (16.72 MB)

 Trainable params: 332,305 (1.27 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

#### 6. Loss objects and custom Model sub class

In [97]:
# Per-sample sparse categorical crossentropy loss with masking for invalid labels (no reduction)
ce_age = keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction="none")
ce_gender = keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction="none")
ce_emotion = keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction="none")

W_AGE = 1.0
W_GENDER = 1.0
W_EMOTION = 1.0


In [98]:
class MultiTaskModel(keras.Model):
    def __init__(self, core_model, **kwargs):
        super().__init__(**kwargs)
        self.core_model = core_model

        self.age_acc = keras.metrics.SparseCategoricalAccuracy(name="age_acc")
        self.gender_acc = keras.metrics.SparseCategoricalAccuracy(name="gender_acc")
        self.emotion_acc = keras.metrics.SparseCategoricalAccuracy(name="emotion_acc")

    @property
    def metrics(self):
        return [self.age_acc, self.gender_acc, self.emotion_acc]

    def train_step(self, data):
        images, labels = data
        age_true = labels["age"]
        gender_true = labels["gender"]
        emotion_true = labels["emotion"]

        has_age = labels["has_age"]
        has_gender = labels["has_gender"]
        has_emotion = labels["has_emotion"]

        with tf.GradientTape() as tape:
            age_pred, gender_pred, emotion_pred = self.core_model(images, training=True)

            age_loss = ce_age(age_true, age_pred) * has_age
            gender_loss = ce_gender(gender_true, gender_pred) * has_gender
            emotion_loss = ce_emotion(emotion_true, emotion_pred) * has_emotion

            eps = 1e-6
            age_loss = tf.reduce_sum(age_loss) / (tf.reduce_sum(has_age) + eps)
            gender_loss = tf.reduce_sum(gender_loss) / (tf.reduce_sum(has_gender) + eps)
            emotion_loss = tf.reduce_sum(emotion_loss) / (tf.reduce_sum(has_emotion) + eps)

            total_loss = W_AGE*age_loss + W_GENDER*gender_loss + W_EMOTION*emotion_loss

        grads = tape.gradient(total_loss, self.core_model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.core_model.trainable_variables))

        self.age_acc.update_state(age_true, age_pred, sample_weight=has_age)
        self.gender_acc.update_state(gender_true, gender_pred, sample_weight=has_gender)
        self.emotion_acc.update_state(emotion_true, emotion_pred, sample_weight=has_emotion)

        return {
            "loss": total_loss,
            "age_loss": age_loss,
            "gender_loss": gender_loss,
            "emotion_loss": emotion_loss,
            "age_acc": self.age_acc.result(),
            "gender_acc": self.gender_acc.result(),
            "emotion_acc": self.emotion_acc.result()
        }

    def test_step(self, data):
        images, labels = data
        age_true = labels["age"]
        gender_true = labels["gender"]
        emotion_true = labels["emotion"]

        has_age = labels["has_age"]
        has_gender = labels["has_gender"]
        has_emotion = labels["has_emotion"]

        age_pred, gender_pred, emotion_pred = self.core_model(images, training=False)

        age_loss = ce_age(age_true, age_pred) * has_age
        gender_loss = ce_gender(gender_true, gender_pred) * has_gender
        emotion_loss = ce_emotion(emotion_true, emotion_pred) * has_emotion

        eps = 1e-6
        age_loss = tf.reduce_sum(age_loss) / (tf.reduce_sum(has_age) + eps)
        gender_loss = tf.reduce_sum(gender_loss) / (tf.reduce_sum(has_gender) + eps)
        emotion_loss = tf.reduce_sum(emotion_loss) / (tf.reduce_sum(has_emotion) + eps)

        total_loss = W_AGE*age_loss + W_GENDER*gender_loss + W_EMOTION*emotion_loss

        self.age_acc.update_state(age_true, age_pred, sample_weight=has_age)
        self.gender_acc.update_state(gender_true, gender_pred, sample_weight=has_gender)
        self.emotion_acc.update_state(emotion_true, emotion_pred, sample_weight=has_emotion)

        return {
            "loss": total_loss,
            "age_loss": age_loss,
            "gender_loss": gender_loss,
            "emotion_loss": emotion_loss,
            "age_acc": self.age_acc.result(),
            "gender_acc": self.gender_acc.result(),
            "emotion_acc": self.emotion_acc.result()
        }


#### 7. Phase 1 - Training the heads with frozen EfficientNetB0

This stage trains the shared dense + three heads while keeping EfficientNetB0 fixed.

In [99]:
EPOCHS_HEADS = 10

multi_task_model = MultiTaskModel(core_model=model)

multi_task_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3)
)

history_heads = multi_task_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_HEADS
)


Epoch 1/10
910/910 ━━━━━━━━━━━━━━━━━━━━ 725s 766ms/step - age_acc: 0.5763 - age_loss: 1.4464 - emotion_acc: 0.5163 - emotion_loss: 1.2544 - gender_acc: 0.8202 - gender_loss: 0.3950 - loss: 3.0959 - val_age_acc: 0.6124 - val_age_loss: 0.9721 - val_emotion_acc: 0.5983 - val_emotion_loss: 0.7927 - val_gender_acc: 0.8518 - val_gender_loss: 0.2071 - val_loss: 1.9718
Epoch 2/10
910/910 ━━━━━━━━━━━━━━━━━━━━ 707s 774ms/step - age_acc: 0.6118 - age_loss: 1.3735 - emotion_acc: 0.5901 - emotion_loss: 2.3077 - gender_acc: 0.8406 - gender_loss: 0.2956 - loss: 3.9768 - val_age_acc: 0.6148 - val_age_loss: 0.8889 - val_emotion_acc: 0.6048 - val_emotion_loss: 1.1338 - val_gender_acc: 0.8528 - val_gender_loss: 0.1938 - val_loss: 2.2165
Epoch 3/10
910/910 ━━━━━━━━━━━━━━━━━━━━ 703s 731ms/step - age_acc: 0.6256 - age_loss: 0.6023 - emotion_acc: 0.6132 - emotion_loss: 1.5506 - gender_acc: 0.8489 - gender_loss: 0.5223 - loss: 2.6752 - val_age_acc: 0.6224 - val_age_loss: 1.1703 - val_emotion_acc: 0.6011 - val

In [102]:
# Print accuracies from Phase 1
print("Phase 1 - Head Training Results:")
print(f"Age Accuracy: {history_heads.history['age_acc'][-1]:.4f}")
print(f"Gender Accuracy: {history_heads.history['gender_acc'][-1]:.4f}")
print(f"Emotion Accuracy: {history_heads.history['emotion_acc'][-1]:.4f}")

print("\nPhase 1 - Validation Results:")
print(f"Val Age Accuracy: {history_heads.history['val_age_acc'][-1]:.4f}")
print(f"Val Gender Accuracy: {history_heads.history['val_gender_acc'][-1]:.4f}")
print(f"Val Emotion Accuracy: {history_heads.history['val_emotion_acc'][-1]:.4f}")

Phase 1 - Head Training Results:
Age Accuracy: 0.7037
Gender Accuracy: 0.8790
Emotion Accuracy: 0.7228

Phase 1 - Validation Results:
Val Age Accuracy: 0.6283
Val Gender Accuracy: 0.8553
Val Emotion Accuracy: 0.6464


#### 8. Phase 2 - Fine-tune top EfficientNetB0 blocks

* Freezing BN layers is standard to keep their statistics stable during fine‑tuning.
* Lower LR prevents destroying pretrained weights while still adapting to your tasks.

In [103]:
# Choose how much of EfficientNetB0 to unfreeze:
# Here we unfreeze the last N layers (you can tune this).
fine_tune_at = int(len(base_model.layers) * 0.7)  # unfreeze top 30% of layers

for layer in base_model.layers[fine_tune_at:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

# Re-compile with lower learning rate for fine-tuning
multi_task_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
)

history_finetune = multi_task_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_FINETUNE,
)

Epoch 1/10
910/910 ━━━━━━━━━━━━━━━━━━━━ 712s 747ms/step - age_acc: 0.7191 - age_loss: 1.2505 - emotion_acc: 0.7465 - emotion_loss: 0.2486 - gender_acc: 0.8819 - gender_loss: 0.2592 - loss: 1.7583 - val_age_acc: 0.6458 - val_age_loss: 1.0383 - val_emotion_acc: 0.6843 - val_emotion_loss: 0.6937 - val_gender_acc: 0.8628 - val_gender_loss: 0.1977 - val_loss: 1.9297
Epoch 2/10
910/910 ━━━━━━━━━━━━━━━━━━━━ 858s 940ms/step - age_acc: 0.7404 - age_loss: 0.5036 - emotion_acc: 0.8027 - emotion_loss: 1.6409 - gender_acc: 0.8931 - gender_loss: 0.0973 - loss: 2.2418 - val_age_acc: 0.6498 - val_age_loss: 0.9533 - val_emotion_acc: 0.7129 - val_emotion_loss: 0.8917 - val_gender_acc: 0.8628 - val_gender_loss: 0.1931 - val_loss: 2.0381
Epoch 3/10
910/910 ━━━━━━━━━━━━━━━━━━━━ 822s 898ms/step - age_acc: 0.7605 - age_loss: 0.3300 - emotion_acc: 0.8406 - emotion_loss: 0.3449 - gender_acc: 0.9003 - gender_loss: 0.3244 - loss: 0.9993 - val_age_acc: 0.6413 - val_age_loss: 0.9209 - val_emotion_acc: 0.7219 - val

#### 9. Simple inference helper

In [105]:
# Inference helper on a single image

age_label_map = {i: i for i in range(NUM_AGE_CLASSES)}  # or bins
gender_label_map = {0: "male", 1: "female"}             # adjust if needed
emotion_label_map = {i: f"class_{i}" for i in range(NUM_EMOTION_CLASSES)}  # replace later

def predict_on_image(img_path):
    img = load_and_preprocess(img_path)
    img = tf.expand_dims(img, axis=0)

    # Use the full multitask model
    age_logits, gender_logits, emotion_logits = model(img, training=False)

    age_pred = tf.argmax(age_logits, axis=-1).numpy()[0]
    gender_pred = tf.argmax(gender_logits, axis=-1).numpy()[0]
    emotion_pred = tf.argmax(emotion_logits, axis=-1).numpy()[0]

    print("Pred age bin:", age_label_map[age_pred])
    print("Pred gender:", gender_label_map[gender_pred])
    print("Pred emotion:", emotion_label_map[emotion_pred])


# Example:
example_path = train_df.iloc[0]["image_path"]
# print("Example image path:", example_path)
# printing the image object to verify it's correct
img = load_and_preprocess(example_path)
print("Loaded image shape:", img.shape)
predict_on_image(example_path)

Loaded image shape: (224, 224, 3)
Pred age bin: 3
Pred gender: male
Pred emotion: class_4


In [106]:
# Phase 1 accuracies
print("Phase 1 - Final Epoch:")
print(f"Age Accuracy: {history_heads.history['age_acc'][-1]:.4f}")
print(f"Gender Accuracy: {history_heads.history['gender_acc'][-1]:.4f}")
print(f"Emotion Accuracy: {history_heads.history['emotion_acc'][-1]:.4f}")
print(f"Val Age Accuracy: {history_heads.history['val_age_acc'][-1]:.4f}")
print(f"Val Gender Accuracy: {history_heads.history['val_gender_acc'][-1]:.4f}")
print(f"Val Emotion Accuracy: {history_heads.history['val_emotion_acc'][-1]:.4f}")

# Phase 2 accuracies
print("\nPhase 2 - Final Epoch:")
print(f"Age Accuracy: {history_finetune.history['age_acc'][-1]:.4f}")
print(f"Gender Accuracy: {history_finetune.history['gender_acc'][-1]:.4f}")
print(f"Emotion Accuracy: {history_finetune.history['emotion_acc'][-1]:.4f}")
print(f"Val Age Accuracy: {history_finetune.history['val_age_acc'][-1]:.4f}")
print(f"Val Gender Accuracy: {history_finetune.history['val_gender_acc'][-1]:.4f}")
print(f"Val Emotion Accuracy: {history_finetune.history['val_emotion_acc'][-1]:.4f}")

Phase 1 - Final Epoch:
Age Accuracy: 0.7037
Gender Accuracy: 0.8790
Emotion Accuracy: 0.7228
Val Age Accuracy: 0.6283
Val Gender Accuracy: 0.8553
Val Emotion Accuracy: 0.6464

Phase 2 - Final Epoch:
Age Accuracy: 0.8657
Gender Accuracy: 0.9361
Emotion Accuracy: 0.9567
Val Age Accuracy: 0.6390
Val Gender Accuracy: 0.8718
Val Emotion Accuracy: 0.7406
